https://www.rxlist.com/

In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()

# 设置代理ip
ipdata = pd.read_excel('D:/Intern_Project/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
# randomproxy = random.choice(proxieslist)

# 设置消息头
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
}

# params = {
#     'ie': 'UTF-8',
#     'wd': 'Python'
# }


In [3]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
    #  访问百度服务器，获取cookie
    res = requests.get('https://www.rxlist.com/supplements/alpha_a.htm', headers=headers_pc, proxies= randomproxy,)
    # 将cookieJar数据转化为字典，s
    cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

# 1、先获取文章链接列表

In [28]:
def get_term_url(url, tag, letter, headers_pc, randomproxy, cookie_dict):
    results = []
    # pbar.set_description(letter)
    try:
        requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
        s = requests.session()
        s.keep_alive = False # 关闭多余连接

        r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
#             encoding = cchardet.detect(r.content)['encoding']
#             html = r.content.decode(encoding)
        r.encoding = 'utf-8'
        html = r.text
#         if os.path.exists(f"D:/Intern_Project/8_RxList/{tag}_{letter}列表页.html"):   
#             return None
        with open(f'D:/Intern_Project/8_RxList/{tag}_{letter}列表页.html', 'w+', encoding='utf-8') as f:
             f.write(html)

        selector = Selector(text=html)
        termUrl_list = selector.xpath(
            "//div[@id='AZ_container']/div[@class='AZ_results']/ul/li/a")
        #print(len(termUrl_list))
        for termUrl in termUrl_list:
            url = termUrl.xpath('./@href').extract()[0]
            term = termUrl.xpath('./text()').extract()[0]
            # print(term)
            tmp = {
                'term': term,
                'url': url
            }
            results.append(tmp)
            # time.sleep()
        # pbar.update(1)
    except Exception as e:
        print(e)
    return results

In [29]:
def task_monitor(task_list, mt_pool, results):
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                results.extend(result)
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [30]:
letters = [chr(i) for i in range(97, 123)] # 生成26个英文字母的列表

# pbar = tqdm(total=len(letters))
tags = ['drugs', 'supplements', 'drugs_class', 'drugs_comparasion']

for tag in tags:
    results = []
    all_craw_tasks = []
    print(tag)
    headers_pc, randomproxy, cookie_dict = changeparams()
    with ThreadPoolExecutor(max_workers=10) as mt_pool:
        for letter in letters:
            if tag == 'drugs' or 'supplements':
                tag_url = f'https://www.rxlist.com/{tag}/alpha_{letter}.htm'
            if tag == 'drugs_class':
                tag_url = f'https://www.rxlist.com/script/main/alphaidx.asp?p={letter}_rx-class'
            if tag == 'drugs_comparasion':
                tag_url = f'https://www.rxlist.com/script/main/alphaidx.asp?p={letter}_rx-dvd'
            tmp_task = mt_pool.submit(get_term_url, tag_url, tag, letter, headers_pc, randomproxy, cookie_dict)
            all_craw_tasks.append(tmp_task)
        
        task_monitor(all_craw_tasks, mt_pool, results)
        with open(f'D:/Intern_Project/8_RxList/{tag}术语链接列表.json', 'w', encoding='utf-8') as f:
            f.write(json.dumps(results, indent=2, ensure_ascii=False))

drugs
supplements
drugs_class
drugs_comparasion


In [13]:
# 用来测试哪些页面能简单获取到
headers_pc, randomproxy, cookie_dict = changeparams()
            
requests.adapters.DEFAULT_RETRIES = 12 # 增加重连次数
s = requests.session()
s.keep_alive = False # 关闭多余连接

# url = f'https://www.medicinenet.com/diseases_and_conditions/alpha_u.htm'
url = f'https://www.rxlist.com/supplements/alpha_a.htm'
# r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
try:  
    # 外网不能加params？？？
    r = requests.get(url, headers=headers_pc, proxies=randomproxy, cookies=cookie_dict, timeout=10)
    
    
    #             encoding = cchardet.detect(r.content)['encoding']
    #             html = r.content.decode(encoding)
    r.encoding = 'utf-8'
    html = r.text
    with open('D:/Intern_Project/8_RxList/test列表页2.html', 'w+', encoding='utf-8') as f:
        f.write(html)
except Exception as e:
    print(e)

selector = Selector(text=html)
termUrl_list = selector.xpath(
    "//div[@id='AZ_container']/div[@class='AZ_results']/ul/li/a")
    
for termUrl in termUrl_list:
    url = termUrl.xpath('./@href').extract()[0]
    term = termUrl.xpath('./text()').extract()[0]
    # print(term)
    tmp = {
        'term': term,
        'url': url
    }

print(tmp)

{'term': 'Avocado Sugar Extract', 'url': 'https://www.rxlist.com/avocado_sugar_extract/supplements.htm'}
